No need to train the model in this section cause the model is already trained

Another way to use the model for prediction

In [3]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from ultralytics import YOLO
import cv2
import torch

print(YOLO._version)
print(cv2.__version__)
print(torch.__version__)

1
4.12.0
2.9.1+cpu


In [2]:
class SmartInventorySystem:
    def __init__(self, model_path):
        self.model=YOLO(model_path)

    def count_products(self, img_path):
        results=self.model(img_path)
        for r in results:
            r.show()
            class_indices=r.boxes.cls.int().tolist()
            class_names=r.names
            counts={name:0 for name in class_names.values()}
            print(counts)
            print(r.boxes.conf)
            for idx in class_indices:
                counts[class_names[idx]]+=1
        self.log_inventory(counts)
        return counts

    def log_inventory(self, counts):
        for name, count in counts.items():
            print(f"Number of {name} detected: {count}")

In [3]:
model_path='runs/detect/new_train/train/weights/best.pt'
img_path='datasets/Cans detection/test/images/youtube-1_jpg.rf.f2ded39edf3082ce5b487a24bb6dabbc.jpg'

result=SmartInventorySystem(model_path)
products=result.count_products('1.png')


image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 39.2ms
Speed: 1.3ms preprocess, 39.2ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 192)
image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 39.2ms
Speed: 1.3ms preprocess, 39.2ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 192)
{'OranSoda': 0, 'Pepsi': 0, 'Sprite': 0}
tensor([0.9342])
Number of OranSoda detected: 0
Number of Pepsi detected: 0
Number of Sprite detected: 1
{'OranSoda': 0, 'Pepsi': 0, 'Sprite': 0}
tensor([0.9342])
Number of OranSoda detected: 0
Number of Pepsi detected: 0
Number of Sprite detected: 1


Object Detection by Web Cam or Video

In [16]:
def real_time_inventory(model_path, video_path=''):
    model=YOLO(model_path) # trained model

    if video_path:
        cap=cv2.VideoCapture(video_path) # Video
    else:
        cap=cv2.VideoCapture(0) # Webcam

    while True:
        ret, frame= cap.read()
        if not ret:
            break
        # Run detection
        results=model(frame) # Run YOLO inference on the frame
        annotated_frame=results[0].plot() # Visualize the results on the frame

        # count
        print(f"Results: {results[0].boxes}")
        counts={}
        for r in results:
            class_indices=r.boxes.cls.int().tolist()
            class_names=r.names
            counts={name:0 for name in class_names.values()}
            for idx in class_indices:
                counts[class_names[idx]]+=1

        # Display counts on screen
        y_offset = 30
        for product, count in counts.items():
            cv2.putText(annotated_frame, f"{product}: {count}", 
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30
        total_count=0
        for count in counts.values():
            total_count+=count
        cv2.putText(annotated_frame, f"Total products: {total_count}", 
                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset += 30
            
        cv2.imshow('YOLO Real-time Detection', annotated_frame) # # Display the annotated frame
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()


video_path='video2.webm'
real_time_inventory(model_path, video_path)



0: 128x256 1 Sprite, 18.4ms
Speed: 0.7ms preprocess, 18.4ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 256)
Results: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.5923])
data: tensor([[1.3231e+03, 6.4969e+02, 1.4519e+03, 9.5369e+02, 5.9226e-01, 2.0000e+00]])
id: None
is_track: False
orig_shape: (1270, 2560)
shape: torch.Size([1, 6])
xywh: tensor([[1387.5249,  801.6929,  128.8300,  304.0004]])
xywhn: tensor([[0.5420, 0.6313, 0.0503, 0.2394]])
xyxy: tensor([[1323.1099,  649.6927, 1451.9398,  953.6931]])
xyxyn: tensor([[0.5168, 0.5116, 0.5672, 0.7509]])

0: 128x256 1 Sprite, 21.3ms
Speed: 3.7ms preprocess, 21.3ms inference, 1.3ms postprocess per image at shape (1, 3, 128, 256)
Results: ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.7450])
data: tensor([[1.3278e+03, 6.1655e+02, 1.4648e+03, 9.7243e+02, 7.4498e-01, 2.0000e+00]])
id: None
is_track: False
orig_shape: (1260, 2560)
shape:

In [17]:
def real_time_inventory(model_path, video_path=''):
    model=YOLO(model_path) # trained model

    if video_path:
        cap=cv2.VideoCapture(video_path) # Video
    else:
        cap=cv2.VideoCapture(0) # Webcam

    while True:
        ret, frame= cap.read()
        if not ret:
            break
        # Run detection
        results=model(frame) # Run YOLO inference on the frame
        
        # Filter detections with confidence >= 0.70
        filtered_results = []
        for r in results:
            # Get boxes, confidence scores, and class indices
            boxes = r.boxes
            if boxes is not None and len(boxes) > 0:
                confidences = boxes.conf
                class_indices = boxes.cls
                
                # Filter for high confidence detections
                high_conf_mask = confidences >= 0.60
                high_conf_boxes = boxes[high_conf_mask]
                
                # Create a filtered result object
                if len(high_conf_boxes) > 0:
                    filtered_results.append(high_conf_boxes)
        
        # Use filtered results for visualization and counting
        if filtered_results:
            # For visualization - use the original results but only plot high confidence ones
            annotated_frame = frame.copy()
            for boxes in filtered_results:
                # Draw only high confidence boxes
                for box in boxes:
                    # Extract box coordinates and draw manually
                    x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                    conf = box.conf[0].item()
                    cls = box.cls[0].int().item()
                    
                    # Draw rectangle
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    # Draw label with confidence
                    label = f"{model.names[cls]} {conf:.2f}"
                    cv2.putText(annotated_frame, label, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            annotated_frame = frame.copy()

        # count - only high confidence detections
        counts={}
        class_names = model.names
        counts = {name: 0 for name in class_names.values()}
        
        total_high_conf_detections = 0
        for boxes in filtered_results:
            class_indices = boxes.cls.int().tolist()
            for idx in class_indices:
                counts[class_names[idx]] += 1
                total_high_conf_detections += 1

        # Display counts on screen
        y_offset = 30
        for product, count in counts.items():
            cv2.putText(annotated_frame, f"{product}: {count}", 
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30
        
        cv2.putText(annotated_frame, f"Total products: {total_high_conf_detections}", 
                    (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        y_offset += 30
            
        cv2.imshow('YOLO Real-time Detection', annotated_frame) # Display the annotated frame
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 
    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()


video_path='video2.webm'
real_time_inventory(model_path, video_path)


0: 128x256 1 Sprite, 10.9ms
Speed: 1.1ms preprocess, 10.9ms inference, 0.5ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 1 Sprite, 26.2ms
Speed: 1.1ms preprocess, 26.2ms inference, 2.2ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 1 Sprite, 33.0ms
Speed: 0.8ms preprocess, 33.0ms inference, 2.4ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 1 Sprite, 34.5ms
Speed: 2.5ms preprocess, 34.5ms inference, 1.6ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 2 Sprites, 34.3ms
Speed: 1.6ms preprocess, 34.3ms inference, 1.2ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 2 Sprites, 31.1ms
Speed: 0.9ms preprocess, 31.1ms inference, 0.9ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 2 Sprites, 35.5ms
Speed: 1.0ms preprocess, 35.5ms inference, 1.7ms postprocess per image at shape (1, 3, 128, 256)

0: 128x256 2 Sprites, 52.8ms
Speed: 1.1ms preprocess, 52.8ms inference, 2.3ms postprocess per image at shape (1, 3

In [ ]:
from ultralytics import YOLO

model = YOLO(model_path)
metrics = model.val(data='datasets/Cans detection/data.yaml')
print(metrics)

In [23]:
soda={'OranSoda': 20, 'Pepsi': 2, 'Sprite': 2}
sum=soda.values()
su=0
for x in sum:
    su+=x
print(su)


24


In [44]:
# Checking how many images are there
import os

def count(dir):
    img_ext= ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    count=0
    for filename in os.listdir(dir):
        if any(filename.endswith(ext) for ext in img_ext):
                count += 1
    return count

directory = './train/images/'
image_count = count(directory)
print(f'Total images: {image_count}')




Total images: 13685


In [ ]:
# TODOS LIST
# Set a limit to only track object above confidence 0.70

In [3]:
from ultralytics import YOLO

# Load a pre-trained model
model = YOLO('yolov8n.pt')

# Fine-tune the model
model.train(
    data='data.yaml',
    epochs=50,           # Set the number of epochs as needed
    imgsz=320,           # Image size
    batch=4,            # Batch size
    lr0=0.001,           # Initial learning rate
    device='cpu',             # Use GPU if available
    project='runs/detect/trained_model',
)

# Validate on training data
model.val()

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

RuntimeError: Dataset 'data.yaml' error ❌ 'data.yaml' does not exist

# Project Enhancement: Step-by-Step Plan

## 1. Model Performance Comparison

- Fine-tune YOLO on your custom dataset.
- Evaluate both original YOLO and fine-tuned YOLO on the same test images/videos.
- Collect metrics: mAP, precision, recall, F1-score, inference speed.
- Visualize results with tables and plots.

## 2. Back-of-Stock Analysis System

- Use the optimized YOLO model to detect and count products on simulated convenience store shelf images/videos.
- Analyze spatial distribution: map product locations, highlight empty spots.
- Detect out-of-stock or low-stock items in real time.

## 3. Web Dashboard Interface

- Build a simple web dashboard (Flask or Streamlit):
    - Display shelf images with bounding boxes and product counts.
    - Show alerts for out-of-stock items.
    - Present trends over time (stock levels, consumption rates).

## 4. Integration and Demo

- Integrate model, analytics, and dashboard.
- Run a demo with sample shelf images/videos.
- Document results and insights.

---
**You can expand each step with code, visuals, and explanations. Let me know which part you want to start with!**

In [14]:
def simple_model_comparison(original_path, fine_tuned_path, test_images):
    """
    Simple comparison without complex validation
    """
    original_model = SmartInventorySystem(original_path)
    fine_tuned_model = SmartInventorySystem(fine_tuned_path)
    
    comparison_results = []
    
    for img_path in test_images:
        print(f"\n📊 Analyzing {img_path}...")
        
        # Count with original model
        orig_count = original_model.count_products(img_path)
        print(f"Original model detected: {orig_count}")
        
        # Count with fine-tuned model
        fine_count = fine_tuned_model.count_products(img_path)
        print(f"Fine-tuned model detected: {fine_count}")
        
        comparison_results.append({
            'image': img_path,
            'original': orig_count,
            'fine_tuned': fine_count
        })
    
    return comparison_results

# img_path='datasets/Cans detection/test/images/youtube-1_jpg.rf.f2ded39edf3082ce5b487a24bb6dabbc.jpg'
FineTune_model='runs/detect/new_train/train/weights/best.pt'
original_model='yolov8n.pt'

# Usage
test_images = ['img.jpg', '1.png']  # Your test images
comparison_results = simple_model_comparison(original_model, FineTune_model, test_images)

# Display comparison summary
print("\n" + "="*50)
print("📈 COMPARISON SUMMARY")
print("="*50)

for result in comparison_results:
    print(f"\nImage: {result['image']}")
    fine_total = sum(result['fine_tuned'].values())
    orig_total = sum(result['original'].values())
    
    print(f"  Fine-tuned: {fine_total} total products")
    print(f"  Original: {orig_total} total products")
    print(f"  Difference: {fine_total - orig_total}")



📊 Analyzing img.jpg...

image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\img.jpg: 384x640 1 bottle, 50.7ms
Speed: 2.2ms preprocess, 50.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\img.jpg: 384x640 1 bottle, 50.7ms
Speed: 2.2ms preprocess, 50.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
{'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottl

In [33]:
import torch
from ultralytics import YOLO
import cv2
from datetime import datetime
import json
import os

# Fix: ensure built-in sum is not shadowed
sum = __builtins__.sum

class AdvancedInventorySystem:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.inventory_history = []
        self.alert_threshold = 3  # Minimum stock level for alerts
        
    def count_products(self, img_path):
        """Your existing counting function enhanced with analytics"""
        results = self.model(img_path)
        
        # Count products
        product_counts = {}
        detection_details = []
        
        for r in results:
            r.show()
            for box in r.boxes:
                class_id = int(box.cls[0])
                product_name = r.names[class_id]
                confidence = float(box.conf[0])
                
                # Update counts
                product_counts[product_name] = product_counts.get(product_name, 0) + 1
                
                # Store detection details for analytics
                detection_details.append({
                    'product': product_name,
                    'confidence': confidence,
                    'timestamp': datetime.now()
                })
        
        # Log inventory with timestamp
        inventory_record = {
            'timestamp': datetime.now(),
            'products': product_counts.copy(),
            'total_items': sum(product_counts.values()),
            'detection_details': detection_details,
            'image_path': img_path
        }
        
        self.inventory_history.append(inventory_record)
        
        return product_counts, self.inventory_history
    
    def get_inventory_analytics(self):
        """Generate inventory analytics"""
        if not self.inventory_history:
            return {}
        
        current_inventory = self.inventory_history[-1]['products']
        
        analytics = {
            'current_inventory': current_inventory,
            'total_items': sum(current_inventory.values()),
            'inventory_health': self._calculate_inventory_health(current_inventory),
            'low_stock_alerts': self._check_low_stock(current_inventory),
            'trend_analysis': self._analyze_trends(),
            'timestamp': datetime.now()
        }
        
        return analytics
    
    def _calculate_inventory_health(self, inventory):
        """Calculate overall inventory health score"""
        # Define ideal stock levels
        ideal_stock = {
            'Sprite': 10,
            'Pepsi': 8, 
            'OranSoda': 6
        }
        
        health_score = 100
        penalties = 0
        
        for product, ideal in ideal_stock.items():
            current = inventory.get(product, 0)
            
            if current == 0:
                penalties += 30  # Heavy penalty for out-of-stock
            elif current < self.alert_threshold:
                penalties += 20  # Penalty for low stock
            elif current < ideal * 0.5:
                penalties += 10  # Penalty for below 50% ideal
        
        return max(0, health_score - penalties)
    
    def _check_low_stock(self, inventory):
        """Check for low stock alerts"""
        alerts = []
        
        for product, count in inventory.items():
            if count <= self.alert_threshold:
                alerts.append({
                    'product': product,
                    'current_stock': count,
                    'alert_level': 'CRITICAL' if count == 0 else 'LOW',
                    'message': f"{product} is {'out of stock' if count == 0 else 'running low'} ({count} remaining)"
                })
        
        return alerts
    
    def _analyze_trends(self):
        """Analyze inventory trends over time"""
        if len(self.inventory_history) < 2:
            return {"message": "Insufficient data for trend analysis"}
        
        # Calculate consumption rates
        trends = {}
        recent_records = self.inventory_history[-5:]  # Last 5 records
        
        for product in ['Sprite', 'Pepsi', 'OranSoda']:
            product_counts = [record['products'].get(product, 0) for record in recent_records]
            
            if len(product_counts) > 1:
                consumption_rate = product_counts[0] - product_counts[-1]
                trends[product] = {
                    'consumption_rate': max(0, consumption_rate),
                    'current_trend': 'STABLE' if consumption_rate == 0 else 'DECREASING',
                    'suggested_reorder': consumption_rate > 0
                }
        
        return trends


In [42]:
# Example: create an instance of AdvancedInventorySystem
model_path='runs/detect/new_train/train/weights/best.pt'  # Update with your actual weights path
inventory_system = AdvancedInventorySystem(model_path)
# Fix: ensure no variable named 'sum' is shadowing built-in sum
# If you still get TypeError, restart the kernel and rerun all cells
inventory_system.count_products('1.png')
inventory_system.get_inventory_analytics()
inventory_system._analyze_trends()



image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 192)
image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 192)



image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 192)
image 1/1 c:\Users\User\Desktop\Personal\AI\AI_ML\YOLO-based object detection\1.png: 256x192 1 Sprite, 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 192)


{'message': 'Insufficient data for trend analysis'}